#### **Глава 10. Обучение моделей**

В главе 1 вы узнали, что библиотека spaCy включает в себя статистические нейросетевые модели, предобученные для распознавания именованных сущностей, частеречной разметки, синтаксического разбора зависимостей и предсказания семантического подобия.

Но никто не ограничивает нас одними лишь предобученными готовыми моделями. При желании можно обучить модель на собственных примерах, настроив компоненты ее конвейера под потребности своего приложения.


В этой главе обсуждаются вопросы обучения средств распознавания именованных сущностей и синтаксического разбора зависимостей — компонентов конвейера. Их чаще всего приходится настраивать так, чтобы модель отвечала требованиям конкретного сценария использования. 

Кроме того, вы узнаете, как сохранить настроенный компонент конвейера на диск для последующей загрузки его в другом сценарии или модели.

In [1]:
import ru_core_news_lg
import spacy as sp
import sys

from spacy.symbols import ORTH, LEMMA
from IPython.display import Image
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

from spacy import displacy
from IPython.display import Image

from function import *

nlp = sp.load('ru_core_news_lg')

#### **Обучение компонента конвейера модели**
Для настройки под цели и задачи конкретного приложения обучать модель с нуля приходится редко. Обычно можно воспользоваться уже существующей моделью и обновить только определенный компонент конвейера

Этот процесс, как правило, состоит из двух шагов: подготовки обучающих примеров данных (набора предложений с лингвисти- ческой разметкой, подходящих для обучения модели) и передачи их нужному компоненту конвейера

In [2]:
Image(url="images/picture_17.png", width=550, height=450)

Во время обучения библиотека spaCy корректирует весовые коэффициенты модели на основе обучающих примеров данных, стремясь минимизировать погрешность, еще называемую **потерями/функцией потерь (loss)**, предсказаний, выполняемых моделью. Проще говоря, алгоритм вычисляет соотношение токена и метки, определяя вероятность того, что данному токену следует присвоить именно эту метку.

На практике для полноценного обучения компонента модели могут понадобиться сотни или даже тысячи обучающих примеров данных.

**! Прежде чем обучать определенный компонент, необходимо временно отключить все остальные компоненты конвейера модели, чтобы защитить их от ненужных изменений.**

#### **Обучение средства распознавания именованных сущностей**

Представьте, что вы разрабатываете чат-бот для такси, который, само собой, должен правильно распознавать все географические названия в городе и окрестностях. Для этого необходимо обновить систему рас- познавания именованных сущностей модели на основе своих примеров данных. Например, слово Solnce, обозначающее название микрорайона в городе, система должна определять как географическую сущность. В следующих разделах описан способ решения этой задачи.

#### **Определяем, нужно ли обучать средство распознавания именованных сущностей**

In [3]:
doc = nlp(u'Не могли бы вы заехать за мной в Солнце?')

for ent in doc.ents:
    print(ent.text, ent.label_)

print(sp.explain('LOC'))

Солнце LOC
Non-GPE locations, mountain ranges, bodies of water


Получается, что средство распознавания именованных сущностей классифицировало Solnce как местоположение, не являющееся сущно- стью типа **GPE**. Значит, чтобы Solnce распознавалось как сущность типа GPE, средство распознавания нужно обновить. О том, как это сделать, поговорим в следующих разделах.

Однако GPE - не встречается в русской модели spacy

#### **Создание обучающих примеров данных**

In [4]:
train_exams = [
    ('Не могли бы вы прислать такси в Солнце?', {'entities': [(32, 38, 'GPE')] }),
    ('Существует ли фиксированный тариф на проезд в аэропорт из Солнца?', {'entities': [(58, 64, 'GPE')]}),
    ('Сколько времени сейчас занимает ожидание такси?', {'entities': []})
]

#### **Автоматизация процесса создания примеров данных**

Вы уже могли убедиться в том, что создание обучающего набора данных вручную требует немало времени. К тому же оно чревато ошибками, особенно если требуется обработать сотни или тысячи высказываний. Автоматизировать этот утомительный процесс можно с помощью следующего сценария, который быстро создает набор обучающих примеров данных на основе заданного текста:

In [5]:
doc = nlp(u'Не могли бы вы прислать такси в Даунтон? Мне нужно попасть в Москву. Не могли бы вы прислать такси на час позже?')

#f = open("test.txt","rb")
# #contents =f.read()
#doc = nlp(contents.decode('utf8'))
train_exams = []
districts = ['Солнце', 'Гринвальд', 'Даунтон'] 

for sent in doc.sents:
    entities = []
    for token in sent:
        if token.ent_type != 0:
            start = token.idx - sent.start_char
            if token.text in districts:
                entity = (start, start + len(token), 'LOC')
            else:
                entity = (start, start + len(token), token.ent_type_)
            entities.append(entity)

    tpl = (sent.text, {'entities': entities})
    train_exams.append(tpl)

train_exams

[('Не могли бы вы прислать такси в Даунтон?', {'entities': [(32, 39, 'LOC')]}),
 ('Мне нужно попасть в Москву.', {'entities': [(20, 26, 'LOC')]}),
 ('Не могли бы вы прислать такси на час позже?', {'entities': []})]

Для улучшения результатов обучения в обучающий набор необходимо включать не только примеры прочих типов сущностей, но и примеры, не содержащие никаких сущностей. В подразделе «Процесс обучения» на с. 203 обо всем этом рассказано более подробно.

#### **Отключение лишних компонентов конвейера**
Документация spaCy рекомендует перед запуском процесса обуче- ния конкретного компонента конвейера отключать все прочие его компоненты, чтобы модифицировать только тот компонент, который необходимо обновить. 

Следующий код отключает все компоненты конвейера, за исключением средства распознавания именованных сущностей.

In [6]:
nlp.meta['pipeline']
print(nlp.pipe_names)

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [7]:
#other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

#print(nlp.disable_pipes(*other_pipes))

Теперь можно начинать обучение средства распознавания именованных сущностей нахождению новых сущностей, описанных в обучающих примерах данных.

### **Процесс обучения**

Следующий код иллюстрирует обработку обучающих примеров данных по отдельным порциям (так называемым батчам). При этом обучающие примеры демонстрируются модели в различных представлениях во избежание усвоения ненужных обобщений, которые могут встретиться в обучающем корпусе.


Можно было бы создать оптимизатор с помощью метода **nlp.begin_ training()**, но он очищает список типов сущностей. А в этом примере при обновлении уже существующей модели нежелательно, чтобы она «забывала» имеющиеся типы сущностей, поэтому воспользуемся методом **nlp.entity.create_optimizer()**, который создает оптимиза- тор для средства распознавания именованных сущностей без потери имеющегося набора типов сущностей.

Как видим метод entity.create_optimizer() не работает для Русского языка

In [8]:
import random
from spacy.util import minibatch, compounding

'''  
optimizer = nlp.entity.create_optimizer()

for i in range(25):
    random.shuffle(train_exams)
    max_batch_size = 3
    batch_size = compounding(2.0, max_batch_size, 1.001)
    batches = minibatch(train_exams, size=batch_size)
    for batch in batches:
        texts, annotations = zip(*batch)
    
        nlp.update(texts, annotations, sgd=optimizer)

    ner = nlp.get_pipe('ner')
    ner.to_disk('/Users/anastasia/ner')
'''


"  \noptimizer = nlp.entity.create_optimizer()\n\nfor i in range(25):\n    random.shuffle(train_exams)\n    max_batch_size = 3\n    batch_size = compounding(2.0, max_batch_size, 1.001)\n    batches = minibatch(train_exams, size=batch_size)\n    for batch in batches:\n        texts, annotations = zip(*batch)\n    \n        nlp.update(texts, annotations, sgd=optimizer)\n\n    ner = nlp.get_pipe('ner')\n    ner.to_disk('/Users/anastasia/ner')\n"

После загрузки и обучение новая модель должна распозновать соответсвтующие именнованые сущности

#### **Создание нового синтаксического анализатора**


В следующих разделах рассмотрим, как создать свой синтаксический анализатор, приспособленный для конкретной задачи. В частности, обучим анализатор, способный раскрывать в предложении семантические отношения, а не синтаксические зависимости. Семантические отношения (semantic relations) складываются в предложении между значениями слов и фраз.

#### **Понимание входного текста с помощью нестандартного синтаксического разбора зависимостей**

Представьте, что чат-бот должен понимать запросы пользователя, сформулированные на английском языке, и преобразовывать их в SQL-запросы **для передачи в базу данных**. Для этого он производит синтаксический разбор, выделяя смысл и разрезая входной текст на части, которые затем используются при формировании запроса к базе данных.

In [9]:
doc = nlp(u'Найти высокооплачиваемую работу без опыта работы.')
print([(t.text, t.dep_, t.head.text) for t in doc])

[('Найти', 'ROOT', 'Найти'), ('высокооплачиваемую', 'amod', 'работу'), ('работу', 'obj', 'Найти'), ('без', 'case', 'опыта'), ('опыта', 'obl', 'Найти'), ('работы', 'nmod', 'опыта'), ('.', 'punct', 'Найти')]


In [10]:
nice_print(doc)

text       pos_       dep_         tag_      
_______________________________________________________
Найти      VERB       ROOT         VERB      
высокооплачиваемую ADJ        amod         ADJ       
работу     NOUN       obj          NOUN      
без        ADP        case         ADP       
опыта      NOUN       obl          NOUN      
работы     NOUN       nmod         NOUN      
.          PUNCT      punct        PUNCT     
_______________________________________________________


In [11]:
displacy.render(doc, style='dep')

Для удовлетворения требований приложения разметку придется по- менять так, чтобы упростить задачу генерации запросов базы данных. Поэтому необходимо реализовать нестандартный синтаксический анализатор, который отображал бы семантические отношения, а не синтаксические зависимости. В данном случае нам нужна дуга между словами **работа** и **опыт**. В следующих разделах описано, как это сделать.

#### **Выбор используемых типов семантических отношений**


Для соответствия требованиям семантики в список необходимо до- бавить еще один тип — ACTIVITY. Он послужит меткой для слова job в нашем примере предложения. (Хотя и исходного множества типов отношений может быть достаточно, ведь работа обычно ассоциируется с конкретным местом, для которого можно использовать тип PLACE.)

#### **Создание обучающих примеров данных**


Не совсем понятно как использовать для русского - массив данных heads???

In [12]:
TRAINING_DATA = [
    ('найти высокооплачиваемую работу без опыта', {
        'heads': [0, 2, 0, 4, 0],
        'deps': ['ROOT', 'obj', 'ROOT', 'obl', 'ROOT']
    }),
    ('найти хорошие занятия по тренировкам рядом с домом', {
        'heads': [0, 2, 0, 4, 2, 0, 7, 5],
        'deps': ['ROOT', 'obj', 'ROOT', 'nmod', 'obj', 'ROOT', 'obl', 'advmod']
}) ]

In [13]:
doc = nlp(u'найти хорошие занятия по тренировкам рядом с домом')
heads = []
deps = []
for token in doc:
    heads.append(token.head.i)
    deps.append(token.head.dep_)
print(heads)
print(deps)

displacy.render(doc, style='dep')
nice_print(doc)

[0, 2, 0, 4, 2, 0, 7, 5]
['ROOT', 'obj', 'ROOT', 'nmod', 'obj', 'ROOT', 'obl', 'advmod']


text       pos_       dep_         tag_      
_______________________________________________________
найти      VERB       ROOT         VERB      
хорошие    ADJ        amod         ADJ       
занятия    NOUN       obj          NOUN      
по         ADP        case         ADP       
тренировкам NOUN       nmod         NOUN      
рядом      ADV        advmod       ADV       
с          ADP        case         ADP       
домом      NOUN       obl          NOUN      
_______________________________________________________


In [14]:
parser_lst = nlp.pipe_labels['parser']

print(parser_lst)

['ROOT', 'acl', 'acl:relcl', 'advcl', 'advmod', 'amod', 'appos', 'aux', 'aux:pass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'cop', 'csubj', 'csubj:pass', 'dep', 'det', 'discourse', 'expl', 'fixed', 'flat', 'flat:foreign', 'flat:name', 'iobj', 'list', 'mark', 'nmod', 'nsubj', 'nsubj:pass', 'nummod', 'nummod:entity', 'nummod:gov', 'obj', 'obl', 'obl:agent', 'orphan', 'parataxis', 'punct', 'xcomp']


#### **Обучение анализатора**


In [15]:
from spacy.training import Example


In [19]:
d = nlp("найти высокооплачиваемую работу без опыта")
example = Example.from_dict(d, {"heads": [0, 2, 0, 4, 0], 'deps': ['ROOT', 'obj', 'ROOT', 'obl', 'ROOT']})
ex = [example]

In [17]:
from spacy.pipeline import EntityRecognizer
nlp = sp.blank('ru')

parser = nlp.create_pipe("parser")
nlp.add_pipe("parser", first=True)

for text, annotations in TRAINING_DATA:
    for d in annotations.get('deps', []):
        parser.add_label(d)

optimizer = nlp.begin_training() 

import random
for i in range(2):
    random.shuffle(TRAINING_DATA)
    nlp.update(ex, sgd=optimizer)
parser.to_disk('/Users/anastasia/parser')

('L-dep', 0, 9000.0)
('R-dep', 0, 9000.0)
('B-ROOT', 0, 9000.0)
('Gold sent starts?', 1, 0)


ValueError: [E1031] Could not find gold transition - see logs above.